<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Chat about my latest profile posts
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Chat_about_my_lastest_profile_posts.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #linkedin #profile #post #stats #naas_drivers #content #automation #csv

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Last update:** 2023-07-26 (Created: 2023-07-24)

**Description:** This notebook provides a way to chat with Naas MyChatGPT about your latest posts published on your LinkedIn profile. 


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin
import pandas as pd
from datetime import datetime
import naas
import os
import json
try:
    from wordcloud import WordCloud
except:
    !pip install wordcloud --user
    from wordcloud import WordCloud
import matplotlib.pyplot as plt

### Setup Variables
[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: This variable stores the li_at cookie value for the LinkedIn account. This cookie is used to authenticate and access the LinkedIn API.
- `JSESSIONID`: This variable stores the JSESSIONID cookie value for the LinkedIn account. This cookie is used to authenticate and access the LinkedIn API.
- `linkedin_url`: This variable stores the LinkedIn profile URL that will be used as an input for the script.
- `limit`: Number of posts retrieved
- `force_update`: Enforce posts update since beginning
- `cron`: cron params for naas scheduler, for information on changing this setting, please check https://crontab.guru/ for information on the required CRON syntax
- `plugin_name`: It represents the name of the plugin, which in this case is "LinkedIn chat posts".
- `plugin_model`: It specifies the model to be used by the plugin, and in this case, it is "gpt-4".
- `plugin_temperature`: It determines the creativity level of the generated content, with higher values resulting in more diverse outputs. In this case, the temperature is set to 1.
- `plugin_max_tokens`: It sets the maximum number of tokens allowed in the generated content. In this case, the limit is set to 2084 tokens.
- `output_dir`: This variable represents the name of the output directory.
- `csv_file_name`: This variable stores the name of the CSV file that will contain the latest posts.
- `image_file_name`: This variable holds the name of the image file that will display the word cloud.
- `plugin_file_name`: This variable contains the name of the plugin file that will analyze the posts.

In [ ]:
# Inputs
li_at = naas.secret.get("LINKEDIN_LI_AT")
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID")
linkedin_url = "https://www.linkedin.com/in/florent-ravenel/"  # EXAMPLE "https://www.linkedin.com/in/myprofile/"
limit = 5
force_update = False
cron = "0 8 * * *"
plugin_name = "LinkedIn posts analyzer"
plugin_model = "gpt-4"
plugin_temperature = 0
plugin_max_tokens = 2084

# Outputs
output_dir = "linkedin/latest_posts/"
csv_file_name = "posts_data.csv"
image_file_name = "wordcloud.png"
plugin_file_name = "posts_analyzer_plugin.json"

## Model

### Set outputs
Establish the output directory and formulate paths for the output files.

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Get latest posts
Retrieve the most recent posts from LinkedIn, establishing a limit to prevent transferring an overwhelming amount of data to the LLM.

In [ ]:
def get_last_posts(
    li_at,
    JSESSIONID,
    linkedin_url,
    limit,
    file_path,
    force_update
):
    # Init
    output_file_exists = False
    
    # Check if output already exists
    if os.path.exists(file_path):
        output_file_exists = True
    
    # Get last posts
    if not output_file_exists or force_update:
        df = linkedin.connect(li_at, JSESSIONID).profile.get_posts_feed(linkedin_url, limit=limit)
        # Save last posts in CSV
        df.to_csv(file_path, index=False)
        print("💾 Posts successfully saved:", file_path)
    else:
        print("➡️ Output file already exists, data retrieved from CSV. To force update set `force_update = True` in your Setup Variables section")
        df = pd.read_csv(file_path)
    return df

df_posts = get_last_posts(li_at, JSESSIONID, linkedin_url, limit, csv_file_path, force_update)
print("✅ Posts fetched:", len(df_posts))
df_posts.head(limit)

### Create world cloud image
Creating a word cloud is useful as it visually represents the frequency or importance of words in a text, providing a quick and insightful overview of the content.

In [ ]:
# Creating the text variable
text = " ".join(text for text in df_posts.astype(str).TEXT)

# Creating word_cloud with text as argument in .generate() method
word_cloud = WordCloud(
    collocations=False,
    background_color="white",
    width=1200,
    height=600
).generate(text)

# Display the generated Word Cloud
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Save your image in PNG
word_cloud.to_file(image_file_path)
print("💾 Image successfully saved:", image_file_path)

# Share output with naas
image_link = naas.asset.add(image_file_path, params={"inline": True})

### Prepare data for plugin
Refine the dataframe for use in the plugin to prevent passing excessive data and tokens to the LLM.

In [ ]:
def create_plugin_data(df):
    # Keep column
    to_keep = [
        "POST_URL",
        "AUTHOR_NAME",
        "PUBLISHED_DATE",
        "TITLE",
        "TEXT",
        "VIEWS",
        "LIKES",
        "COMMENTS",
        "SHARES",
        "ENGAGEMENT_SCORE"
    ]
    df = df[to_keep]
    
    # Filter
    df = df[df["VIEWS"].astype(int) > 0]
    
    # Multiply ENGAGEMENT_SCORE by 100 and drop the original column
    df["ENGAGEMENT_%"] = df["ENGAGEMENT_SCORE"] * 100
    df = df.drop(columns=["ENGAGEMENT_SCORE"])
    return df.reset_index(drop=True)

data = create_plugin_data(df_posts)
data

### Create prompt for plugin
We used Playground to refined it: https://platform.openai.com/playground?mode=chat&model=gpt-4

In [ ]:
prompt = f"""Act as a Social Media Analyst Asssitant. Your job is to help you unravel the story behind the user LinkedIn posts' performance. 
You can dive deep into the data and gather insights that can help boost the user LinkedIn strategy.
You can help the user understand which posts are getting the most views, likes, comments, and shares. 
You can also analyze the engagement of each post and see how it correlates with different factors. 
When you refer to a post, create href in markdown format so that the user can go to the post you mention by clicking on the markdown href link in a new tab.
But that's not all! You can also help identify trends over time, find out the best time to post, best post to do next, understand the impact of different post types, and much more. 
The possibilities are endless, be creative!
Now, let's get started. Here's the data from the user latest LinkedIn posts: 
{data.to_string()}.
Let's dive in and discover the stories the data is waiting to tell! 
The fist message should be about presenting yourself with maximum 5 bullet points and displaying the worldcloud: {image_link}
Then, wait for the first answer from the user, and then start with a first high level analysis. 
"""
prompt

## Output

### Create MyChatGPT plugin
Plugin must be a JSON file with mandatory keys name, model, temperature, max_tokens and prompt

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": prompt,
}

# Save dict to JSON file
with open(plugin_file_path, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:", plugin_file_path)

### Share plugin with naas.asset
You can now use in your MyChatGPT by copy/pasting the URL after the command `/use `

In [ ]:
naas.asset.add(plugin_file_path, params={"inline": True})

### Add Naas scheduler
Schedule your notebook with the naas scheduler feature

In [ ]:
#naas.scheduler.add(cron=cron)

# to de-schedule this notebook, simply run the following command:
# naas.scheduler.delete()